# Tracking Bulk RNA-seq Nextflow runs

## Background

[Nextflow](https://www.nextflow.io/) is a workflow management system used for orchestrating and executing scientific workflows across different computational environments. Fundamental features include ease of scalability, portability, and reproducibility, as it allows researchers to define complex workflows in a platform-agnostic manner and run them efficiently on various computing infrastructures.

Here, we will demonstrate how to track Nextflow workflow execution and generated biological entities with [lamin](https://lamin.ai/).

## Setup

To run this notebook, you need to load a LaminDB instance that has the `bionty` schema mounted.

Here, we’ll create a test instance (skip if you’d like to run it using your instance):

In [ ]:
!lamin init --storage ./nextflow_rna_seq --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import pandas as pd

ln.settings.verbosity = 3  # show hints

In [ ]:
lb.settings.species = "human"

## Fetching and tracking NGS files

A pipeline run is fundamentally one set of transformations. Hence, we create a {func}`docs:lamindb.Transform` object using Bionty's lookup for the [nf-core fetchngs](https://nf-co.re/fetchngs) pipeline.
`lnschema-bionty` currently does not have a `BFXPipeline` ORM and we therefore access Bionty directly.

In [ ]:
import bionty as bt

bfx_lookup = bt.BFXPipeline().lookup()

We plan on using Version 1.9. Let's find it with our lookup object...

In [ ]:
fetch_ngs_1_9 = bfx_lookup.fetchngs_v1_9
fetch_ngs_1_9

...and create a {func}`docs:lamindb.Transform` object.

In [ ]:
fetch_ngs_1_9_transform = ln.Transform(
    name=fetch_ngs_1_9.name, version=fetch_ngs_1_9.versions, type="pipeline"
).save()

In [ ]:
ln.track(fetch_ngs_1_9_transform)

First, we fetch several FASTQ files using [nf-core fetchngs](https://nf-co.re/fetchngs). For simplicity, we use the `test` profile which has a pre-defined small set of FASTQ files. Later, we'll use them as input for [nf-core rna-seq](https://nf-co.re/rnaseq).

In [ ]:
!nextflow run nf-core/fetchngs -r 1.9 --nf_core_pipeline=rnaseq -profile test,docker --outdir fetchngs-results -resume

In [ ]:
!tree fetchngs-results

The pipeline run results in several FASTQ files with associated md5 sum files, metadata files, samplesheets, and execution reports. We ingest all files with Lamin.

In [ ]:
fetchngs_results = ln.File.from_dir("fetchngs-results")
ln.save(fetchngs_results)

Now that all output files or our run are ingested, we can access them easily and work with them.
Lamin tracks data lineage:

In [ ]:
execution_report_file = ln.File.select(key__icontains="execution_report").one()
execution_report_file.view_lineage()

We cna now easily examine for example the execution report:

In [ ]:
import shutil
from IPython.display import IFrame

# Copying file to a directory accessible by the IPython Tornado web server
shutil.copy(execution_report_file.stage(), "./execution_report.html")
IFrame(src="execution_report.html", width=1000, height=600)

Alternatively, we can calculate various metrics of interest for the downloaded FASTQ files.

In [ ]:
a_fastq_file = ln.File.select(
    key__icontains="fetchngs-results/fastq", suffix=".fastq.gz"
).first()
a_fastq_file

In [ ]:
import gzip
from Bio import SeqIO

total_gc_count = total_bases = 0

with gzip.open(a_fastq_file.stage(), "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        sequence = str(record.seq)
        gc_count = sequence.count("G") + sequence.count("C")
        total_gc_count += gc_count
        total_bases += len(sequence)

total_gc_content = (total_gc_count / total_bases) * 100

print(f"Total GC Content: {total_gc_content:.2f}%")

In [ ]:
a_fastq_file.view_lineage()

## Analysing raw FASTQ files and generating a count table

[nf-core rna-seq](https://nf-co.re/rnaseq/3.12.0) is arguably one of the most popular pipelines for bulk RNA sequencing using STAR, RSEM, HISAT2 or Salmon with gene/isoform counts and extensive quality control.

First, we create a new Transform object for our pipeline run.

In [ ]:
rna_seq_3_11_2 = bfx_lookup.rnaseq_v3_11_2
rna_seq_3_11_2

In [ ]:
rna_seq_3_11_2_transform = ln.Transform(
    name=rna_seq_3_11_2.name, version=rna_seq_3_11_2.versions, type="pipeline"
).save()

In [ ]:
ln.track(rna_seq_3_11_2_transform)

Next, we run the pipeline with its test dataset and track output files and features with Lamin.

In [ ]:
!nextflow run nf-core/rnaseq -r 3.11.2 -profile test,docker --outdir rna-seq-results -resume

As a first step, we ingest all results from the pipeline run.

In [ ]:
multiqc_results = ln.File.from_dir("rna-seq-results/multiqc/")
ln.save(multiqc_results)

In [ ]:
multiqc_file = ln.File.select(key__icontains="multiqc_report.html").one()
multiqc_file

Let's examine the multiqc report:

In [ ]:
import shutil
from IPython.display import IFrame

# Copying file to a directory accessible by the IPython Tornado web server
shutil.copy(multiqc_file.stage(), "./multiqc_report.html")
IFrame(src="multiqc_report.html", width=1000, height=600)

We further ingest the merged Salmon gene counts since we plan on working further with the count table:

In [ ]:
salmon_gene_table = ln.File("rna-seq-results/salmon/salmon.merged.gene_counts.tsv")
ln.save(salmon_gene_table)

In [ ]:
gene_counts_df = pd.read_csv(salmon_gene_table.stage(), sep="\t")

In [ ]:
print(f"Number of genes: {len(gene_counts_df)}")
print(
    f"Number of samples: {len(gene_counts_df.columns) - 1}"
)  # Subtract 1 to exclude the gene column
print(f"Mean expression: {gene_counts_df.iloc[:, 2:].mean()}")
print(f"Median expression: {gene_counts_df.iloc[:, 2:].median()}")
print(f"Expression variance: {gene_counts_df.iloc[:, 2:].var()}")

In [ ]:
feature_set_genes = ln.FeatureSet.from_values(
    gene_counts_df["gene_name"], lb.Gene.symbol
)
feature_set_genes.save()
salmon_gene_table.feature_sets.add(feature_set_genes)

The dataset contains yeast samples and our species is set to human. Hence, a lot of gene records are being created. Bionty will soon also support yeast genes.

## Conclusion

TODO